In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import max_error, r2_score, mean_absolute_error
import mlflow
import os

In [32]:
"""
Descarga de Archivos desde Google Drive sin Autenticación

Este script permite descargar archivos específicos desde Google Drive directamente a una ubicación local,
utilizando el ID de Google Drive del archivo (el archivo debe ser accesible para cualquier persona que tenga la URL).

El script procesa un archivo de texto ('archivos_info.txt') que contiene las IDs de Google Drive de los archivos,
los nombres con los que se desean guardar localmente, y las rutas locales de almacenamiento, todo separado por comas.

Formato esperado de 'archivos_info.txt':
ID_de_Google_Drive,Nombre_Archivo_Local,Ruta_Local

Ejemplo:
1hGvKmNAkK...,mi_archivo.txt,./descargas

Requisitos:
- Módulo 'requests' instalado en el entorno Python donde se ejecute este script.



"""

import requests
import os


def descargar_archivo_directo(id_archivo, directorio_destino, archivo_destino):
    """
    Descarga un archivo directamente desde Google Drive y lo guarda localmente.

    Parámetros:
    - id_archivo (str): ID del archivo en Google Drive.
    - directorio_destino (str): Ruta del directorio local donde se guardará el archivo.
    - archivo_destino (str): Nombre deseado para el archivo en local

    Devuelve:
    - archivo_destino (str): Nombre del archivo guardado.
    - ruta_completa (str): Ruta completa del archivo guardado.
    """
    # Construye la URL de descarga directa utilizando el ID del archivo
    url = f"https://drive.google.com/uc?export=download&id={id_archivo}"

    # Realiza la petición HTTP GET para descargar el archivo
    respuesta = requests.get(url, allow_redirects=True)

    # Comprueba que el directorio destino existe, si no, lo crea
    os.makedirs(directorio_destino, exist_ok=True)

    # Construye la ruta completa donde se guardará el archivo en local
    ruta_completa = os.path.join(directorio_destino, archivo_destino)

    # Guarda el contenido del archivo descargado en local
    with open(ruta_completa, 'wb') as archivo:
        archivo.write(respuesta.content)


    return archivo_destino, ruta_completa


def procesar_archivo_info(ruta_archivo_info):
    """
    Procesa un archivo de texto que contiene información sobre los archivos a descargar.

    Parámetros:
    - ruta_archivo_info (str): Ruta del archivo de texto que contiene los IDs de Google Drive,
                               los nombres de los archivos locales y las rutas locales.

    Devuelve:
    - Una lista de tuplas con el ID de Google Drive, el nombre local del archivo, y la ruta local.
    """
    archivos_info = []
    with open(ruta_archivo_info, 'r') as archivo:
        for linea in archivo:
            id_archivo, nombre_archivo, directorio_destino = linea.strip().split(',')
            archivos_info.append((id_archivo, nombre_archivo, directorio_destino))
    return archivos_info


In [33]:
# Ruta al archivo que contiene la información de los archivos a descargar
ruta_archivo_info = "..\info_archivos_GDrive_solo_entr.txt"

# Obtenemos la lista con la info de los archivos del fichero
archivos_a_descargar = procesar_archivo_info(ruta_archivo_info)

# Descargamos cada archivo de la lista
for id_archivo, nombre_archivo, directorio_destino in archivos_a_descargar:
    nombre_archivo_descargado, ruta_archivo_guardado = descargar_archivo_directo(id_archivo, directorio_destino, nombre_archivo)
    print(f"Archivo {nombre_archivo_descargado} guardado en: {ruta_archivo_guardado}")

Archivo datos_preparados_solo_entr.parquet guardado en: ../Downloads\datos_preparados_solo_entr.parquet


In [34]:
df = pd.read_parquet(ruta_archivo_guardado, engine='pyarrow')
df.describe()

,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Tackles,...,nationality_The Gambia,nationality_Türkiye,nationality_Ukraine,nationality_United States,nationality_Uruguay,nationality_Venezuela,nationality_Wales,nationality_Zambia,nationality_Zimbabwe,marketValue
count,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,...,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2.251000e+03
mean,1271.695691,1.564638,1.063527,3.069303,0.168814,0.629276,76.128387,0.946246,0.622834,0.882719,...,0.000444,0.002665,0.001333,0.003554,0.028432,0.003998,0.000888,0.000444,0.000444,8.160329e+06
std,969.241058,2.906800,1.737932,2.958017,0.430942,0.604061,13.308598,0.962542,1.244505,0.693640,...,0.021077,0.051571,0.036490,0.059522,0.166240,0.063119,0.029801,0.021077,0.021077,1.188624e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000e+04
25%,360.500000,0.000000,0.000000,1.000000,0.000000,0.200000,70.950000,0.300000,0.000000,0.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+06
50%,1157.000000,0.000000,0.000000,2.000000,0.000000,0.500000,78.300000,0.700000,0.000000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000e+06
75%,2040.500000,2.000000,2.000000,5.000000,0.000000,0.900000,84.200000,1.300000,1.000000,1.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.500000e+06
max,3420.000000,27.000000,16.000000,15.000000,3.000000,4.500000,100.000000,8.700000,13.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.500000e+07


In [35]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("regresion_lineal")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1713193978222, experiment_id='3', last_update_time=1713193978222, lifecycle_stage='active', name='regresion_lineal', tags={}>

# Con Variables Elegidas

In [36]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow

X = df[['Mins', 'Assists' ,'MotM', 'Titularidades', 'Equipo_pos', '1_año_anterior',
 '2_año_anterior', '3_año_anterior', '4_año_anterior']]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)
# Crear el modelo de regresión lineal
model = LinearRegression()

# Realizar validación cruzada con 10 folds
cv_results = cross_validate(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)

# Extraer los resultados de la validación cruzada
train_mae_scores = -cv_results['train_score']
val_mae_scores = -cv_results['test_score']

# Calcular el error absoluto medio promedio en entrenamiento y validación
mean_train_mae = np.mean(train_mae_scores)
mean_val_mae = np.mean(val_mae_scores)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="FINAL MODEL WITH FEATURE SELECTION"):
    # Registrar los resultados de cada capa de la validación cruzada en MLflow
    for i, (train_score, val_score) in enumerate(zip(train_mae_scores, val_mae_scores), start=1):
        mlflow.log_metric(f"train_mae_fold_{i}", train_score)
        mlflow.log_metric(f"val_mae_fold_{i}", val_score)

    # Calcular la media de los resultados de validación
    mean_val_mae_cv = np.mean(val_mae_scores)

    # Registrar la media de los resultados de validación en MLflow
    mlflow.log_metric("mean_val_mae_cv", mean_val_mae_cv)

    print("Mean Train MAE across folds:", mean_train_mae)
    print("Mean Validation MAE across folds:", mean_val_mae_cv)

    # Entrenar el modelo final con todos los datos de entrenamiento
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)
    # Calcular el coeficiente de determinación (R^2) en el conjunto de prueba
    r2 = r2_score(y_test, y_test_pred)
    
    # Calcular el número de observaciones y características
    n = X_test.shape[0]
    p = X_test.shape[1]
    
    # Calcular el R^2 ajustado
    r2_adjusted = 1 - ((1 - r2) * (n - 1) / (n - p - 1))



    # Calcular el MAE en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el MAE en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)
        
    # Registrar el R^2 ajustado en MLflow
    mlflow.log_metric("test_r2_adjusted", r2_adjusted)
    print("Test R^2 adjusted:", r2_adjusted)
    mlflow.log_params({
        "features_selected": ['Mins', 'Assists', 'MotM', 'Titularidades', 'Equipo_pos', '1_año_anterior', '2_año_anterior', '3_año_anterior', '4_año_anterior']
    })
    print("Test MAE:", test_mae)

Mean Train MAE across folds: 2815439.949276029
Mean Validation MAE across folds: 2848775.3199543413
Test R^2 adjusted: 0.820062930230275
Test MAE: 2787930.650162342


# Sin Seleccion de Atributos

In [37]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import mlflow

X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

# Crear el modelo de regresión lineal
model = LinearRegression()

# Realizar validación cruzada con 10 folds
cv_results = cross_validate(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)

# Extraer los resultados de la validación cruzada
train_mae_scores = -cv_results['train_score']
val_mae_scores = -cv_results['test_score']

# Calcular el error absoluto medio promedio en entrenamiento y validación
mean_train_mae = np.mean(train_mae_scores)
mean_val_mae = np.mean(val_mae_scores)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="EVALUACIÓN CON TODAS LAS VARIABLES"):
    # Registrar los resultados de cada capa de la validación cruzada en MLflow
    for i, (train_score, val_score) in enumerate(zip(train_mae_scores, val_mae_scores), start=1):
        mlflow.log_metric(f"train_mae_fold_{i}", train_score)
        mlflow.log_metric(f"val_mae_fold_{i}", val_score)

    # Calcular la media de los resultados de entrenamiento y validación
    mean_train_mae_cv = np.mean(train_mae_scores)
    mean_val_mae_cv = np.mean(val_mae_scores)

    # Registrar la media de los resultados de entrenamiento y validación en MLflow
    mlflow.log_metric("mean_train_mae_cv", mean_train_mae_cv)
    mlflow.log_metric("mean_val_mae_cv", mean_val_mae_cv)

    print("Mean Train MAE across folds:", mean_train_mae_cv)
    print("Mean Validation MAE across folds:", mean_val_mae_cv)

    # Entrenar el modelo final con todos los datos de entrenamiento
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_test_pred = model.predict(X_test)

    # Calcular el MAE en el conjunto de prueba
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Registrar el MAE en el conjunto de prueba en MLflow
    mlflow.log_metric("test_mae", test_mae)

    print("Test MAE:", test_mae)


Mean Train MAE across folds: 2735563.2582674064
Mean Validation MAE across folds: 3075473.9950745413
Test MAE: 3015336.5134360217
